In [3]:
import mxnet as mx
from mxnet import nd, autograd as ag, gluon
from get_data import get_data

In [8]:
# 数据

num_input = 2
num_output = 1
num_examples = 1000
batch_size = 10

train_data,train_label = get_data(W=[3,-6],b=9,num_data=num_examples)
X = nd.array(train_data)
Y = nd.array(train_label)

batch_size = 10
train_data = gluon.data.DataLoader(
    gluon.data.ArrayDataset(X,Y),
    batch_size=batch_size,
    shuffle=True)

In [9]:
# 算法

net = gluon.nn.Dense(1, in_units=2) # input 2维，output 1维
net.collect_params().initialize(mx.init.Normal(sigma=1.))

squire_loss = gluon.loss.L2Loss()

trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':.01})

In [13]:
# 训练

epochs = 5
model_ctx = mx.cpu()

for e in range(epochs):
    cumulative_loss = 0
    for i,(data,label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        with ag.record():
            output = net(data)
            loss = squire_loss(output,label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.mean(loss).asscalar()
    print("%s: %s"%(e,cumulative_loss/num_examples))

print(net.weight.data(), net.bias.data())

0: 0.0005362845647614449
1: 0.00053641154745128
2: 0.0005363112081540749
3: 0.0005362713750218973
4: 0.0005362537732580677

[[ 3.0012105 -6.000957 ]]
<NDArray 1x2 @cpu(0)> 
[9.00212]
<NDArray 1 @cpu(0)>
